In [ ]:
import yfinance as yf

ticker = yf.Ticker("PETR4.SA")
hist = ticker.history(period="5y")
inicio = hist["Close"].iloc[0]
fim = hist["Close"].iloc[-1]
anos = 5
cagr = ((fim / inicio) ** (1 / anos)) - 1
print(f"CAGR: {cagr:.2%}")

CAGR: 36.22%


In [1]:
import yfinance as yf

# Exemplo com ações brasileiras (use .SA no final do ticker)
taesa = yf.Ticker("TAEE11.SA")
itausa = yf.Ticker("ITSA4.SA")

# Pega as recomendações de analistas (se disponível)
recomendacoes_taesa = taesa.recommendations
recomendacoes_itausa = itausa.recommendations

print("Recomendações para TAESA:")
print(recomendacoes_taesa.tail())  # Mostra as últimas recomendações

print("\nRecomendações para ITAÚSA:")
print(recomendacoes_itausa.tail())


Recomendações para TAESA:
  period  strongBuy  buy  hold  sell  strongSell
0     0m          0    0     4     5           4
1    -1m          0    0     4     5           4
2    -2m          0    0     5     5           3

Recomendações para ITAÚSA:
  period  strongBuy  buy  hold  sell  strongSell
0     0m          2    5     0     0           0
1    -1m          2    5     0     0           0
2    -2m          2    5     0     0           0


## Validacao Divivendo 12M e 5anos

In [8]:
import yfinance as yf
import pandas as pd
from pandas.tseries.offsets import DateOffset
from IPython.display import display, Markdown

def analisar_dividendos(ticker: str):
    """
    Calcula e exibe os dividendos de um ticker nos últimos 12 meses e 5 anos.
    """
    try:
        # Baixa os dados da ação
        stock = yf.Ticker(ticker)
        dividends = stock.dividends

        # Se não houver dividendos, encerra a função.
        if dividends.empty:
            display(Markdown(f"### Análise de Dividendos para: `{ticker}`\n\nNão foram encontrados dados de dividendos para este ativo."))
            return

        # Converte para DataFrame e organiza
        df_dividends = pd.DataFrame(dividends, columns=['Dividends']).sort_index()

        # Define as datas de início e fim para os cálculos
        # A data final é hoje e a inicial é X períodos no passado.
        end_date = pd.Timestamp.now(tz='America/Sao_Paulo')
        start_date_12m = end_date - DateOffset(years=1)
        start_date_5y = end_date - DateOffset(years=5)

        # Filtra o DataFrame para os períodos desejados
        # Apenas dividendos pagos a partir de 5 anos atrás são considerados
        df_dividends_5y_period = df_dividends[df_dividends.index >= start_date_5y]

        # Calcula dividendos dos últimos 5 anos
        div_5y_total = df_dividends_5y_period['Dividends'].sum()

        # Calcula dividendos dos últimos 12 meses a partir do df já filtrado dos 5 anos
        div_12m_total = df_dividends_5y_period[df_dividends_5y_period.index >= start_date_12m]['Dividends'].sum()

        # Exibição dos resultados com Markdown para melhor visualização
        display(Markdown(f"### Análise de Dividendos para: `{ticker}`"))
        display(Markdown(f"**Dividendos Totais (Últimos 12 Meses):** **R$ {div_12m_total:.2f}**"))
        display(Markdown(f"**Dividendos Totais (Últimos 5 Anos):** **R$ {div_5y_total:.2f}**"))

        # Validação e Insights
        if div_5y_total > 0:
            percentual = (div_12m_total / div_5y_total) * 100
            display(Markdown(f"**Insight:** Os dividendos do último ano representam **{percentual:.1f}%** do total pago nos últimos 5 anos."))
            if percentual > 25: # Um valor médio esperado seria 20% (1/5)
                 display(Markdown("> **Projeção:** O pagamento no último ano foi **acima da média** do período, indicando uma possível aceleração na distribuição de proventos."))
            elif percentual < 15:
                 display(Markdown("> **Projeção:** O pagamento no último ano foi **abaixo da média** do período. Vale investigar se foi um evento pontual ou uma mudança na política de dividendos."))

    except Exception as e:
        display(Markdown(f"### Erro ao analisar `{ticker}`\n\nOcorreu um erro: {e}"))

# Exemplo de uso com o ticker do seu código
analisar_dividendos("petr4.SA")

### Análise de Dividendos para: `petr4.SA`

**Dividendos Totais (Últimos 12 Meses):** **R$ 3.51**

**Dividendos Totais (Últimos 5 Anos):** **R$ 28.92**

**Insight:** Os dividendos do último ano representam **12.2%** do total pago nos últimos 5 anos.

> **Projeção:** O pagamento no último ano foi **abaixo da média** do período. Vale investigar se foi um evento pontual ou uma mudança na política de dividendos.

## API EODHD

In [ ]:
api_key = "6865878bb74f39.03100329"

## finnhub

In [ ]:
api_key = "d1iogt1r01qhbuvra280d1iogt1r01qhbuvra28g"

In [13]:
import finnhub
import pandas as pd
from datetime import datetime, timedelta

# Substitua pela sua chave API real obtida em https://finnhub.io/
api_key = "d1iogt1r01qhbuvra280d1iogt1r01qhbuvra28g"

# Configura o cliente Finnhub
finnhub_client = finnhub.Client(api_key=api_key)

# Define o período de 5 anos
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)  # Aproximadamente 5 anos

# Obtém dados de dividendos
symbol = "TAEE4.SA"  # Ticker da Taesa na B3
try:
    dividends = finnhub_client.stock_dividends(symbol, start_date.strftime("%Y-%m-%d"), end_date.strftime("%Y-%m-%d"))
    print("Resposta bruta da API:", dividends)

    # Processa os dividendos, se existirem
    if 'dividends' in dividends:
        df_dividends = pd.DataFrame(dividends['dividends'])
        if not df_dividends.empty:
            print("Dividendos encontrados:")
            print(df_dividends[['date', 'amount']])
        else:
            print("Nenhum dividendo encontrado no período.")
    else:
        print("Chave 'dividends' não encontrada na resposta. Estrutura da resposta:", dividends)

except Exception as e:
    print(f"Erro na requisição: {str(e)}")

Erro na requisição: FinnhubAPIException(status_code: 403): You don't have access to this resource.
